In [ ]:
import pandas as pd
import openai
import time

In [ ]:
OPENAI_API_KEY = "sk-proj..."

client = openai.Client(api_key=OPENAI_API_KEY)

In [ ]:
def find_domain_openai(company_name):
    prompt = f"Find the official domain name of the company '{company_name}'."
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": (
                    "You are a helpful assistant. Only reply with the domain name of the company, "
                    "without any sentence, formatting, or explanation. Example: keplercheuvreux.com"
                    )
                },
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        reply = response.choices[0].message.content.strip()
        
        for word in reply.split():
            if word.startswith("http"):
                return word.strip().rstrip(".")
        return reply

    except Exception as e:
        print(f"Error for {company_name}: {e}")
        return None

In [ ]:
def enrich_domains_with_openai(input_csv, output_csv):
    df = pd.read_csv(input_csv)
    
    for idx, row in df.iterrows():
        if pd.isna(row["Domains"]) and pd.notna(row["Record"]):
            company_name = row["Record"]
            print(f"→ Looking up: {company_name}")
            domain = find_domain_openai(company_name)
            if domain:
                df.at[idx, "Domains"] = domain
                print(f"OK — {company_name} → {domain}")
            else:
                print(f"✕ Could not find domain for {company_name}")
            time.sleep(1)  # Rate limiting
    
    df.to_csv(output_csv, index=False)
    print(f"\nEnriched CSV saved to {output_csv}")

In [ ]:
input_csv = "~/Downloads/sales_contacted.csv"

enrich_domains_with_openai(input_csv, "~/Downloads/enriched_companies.csv")